<a href="https://colab.research.google.com/github/Namithakrt/Title-Generator/blob/main/Title_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import string
import numpy as np
import json

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [ ]:
import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [ ]:
#load all the datasets
df1 = pd.read_csv('USvideos.csv')
df2 = pd.read_csv('CAvideos.csv')
df3 = pd.read_csv('GBvideos.csv')

In [ ]:
#load the datasets containing the category names
data1 = json.load(open('US_category_id.json'))
data2 = json.load(open('CA_category_id.json'))
data3 = json.load(open('GB_category_id.json'))

In [ ]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

In [ ]:
#create a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

In [ ]:
#join the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

In [ ]:
#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()


In [ ]:
#generating sequence

In [ ]:
#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1

  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [ ]:
#padding sequences

In [ ]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len-1
  model = Sequential()

  # Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))

  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))

  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')

  return model
model = create_model(max_sequence_len, total_words)

In [ ]:
model.fit(predictors, label, epochs=30, verbose=1)

Epoch 1/30
2202/2202 [==============================] - 132s 59ms/step - loss: 7.9540
Epoch 2/30
2202/2202 [==============================] - 132s 60ms/step - loss: 7.1337
Epoch 3/30
2202/2202 [==============================] - 128s 58ms/step - loss: 6.6326
Epoch 4/30
2202/2202 [==============================] - 129s 58ms/step - loss: 6.2056
Epoch 5/30
2202/2202 [==============================] - 131s 59ms/step - loss: 5.8119
Epoch 6/30
2202/2202 [==============================] - 131s 60ms/step - loss: 5.4512
Epoch 7/30
2202/2202 [==============================] - 130s 59ms/step - loss: 5.1342
Epoch 8/30
2202/2202 [==============================] - 131s 60ms/step - loss: 4.8444
Epoch 9/30
2202/2202 [==============================] - 134s 61ms/step - loss: 4.5812
Epoch 10/30
2202/2202 [==============================] - 129s 59ms/step - loss: 4.3438
Epoch 11/30
2202/2202 [==============================] - 131s 60ms/step - loss: 4.1210
Epoch 12/30
2202/2202 [=============================

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for j in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)

    output_word = ""
    for word,index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text +=" "+output_word
  return seed_text.title()

In [ ]:
print(generate_text("Game", 5, model, max_sequence_len))

1/1 [==============================] - 0s 25ms/step
Game Of Thrones Season 2 Trailer


In [ ]:
print(generate_text("friends", 5, model, max_sequence_len))

1/1 [==============================] - 0s 31ms/step
Friends Of The Worst Official Trailer


In [ ]:
print(generate_text("simple", 5, model, max_sequence_len))

1/1 [==============================] - 0s 44ms/step
Simple Life Hacks Mrgear Is These
